In [5]:
!pip install streamlit
!pip install img2vec_pytorch
!pip install scikit-learn



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import pickle
import numpy as np

from img2vec_pytorch import Img2Vec
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score

In [7]:
img2vec = Img2Vec()

data_dir = './dataset'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')



data = {}
for j, dir_ in enumerate([train_dir, val_dir]):
    features = []
    labels = []
    for category in os.listdir(dir_):
        for img_path in os.listdir(os.path.join(dir_, category)):
            img_path_ = os.path.join(dir_, category, img_path)
            
            img = Image.open(img_path_).convert('RGB')

            img_features = img2vec.get_vec(img)

            features.append(img_features)
            labels.append(category)

    data[['training_data', 'validation_data'][j]] = features
    data[['training_labels', 'validation_labels'][j]] = labels
    

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
from matplotlib import pyplot as plt
data_dir = './dataset/train'

class_names = sorted(os.listdir(data_dir))
nrows = len(class_names)
ncols = 6
plt.figure(figsize=(ncols*1.5, nrows*1.5))
for row in range(nrows):
    class_name = class_names[row]
    img_paths = [os.path.join(train_dir, class_name, filename)
        for filename in os.listdir(os.path.join(train_dir, class_name))]
    for col in range(ncols):
        plt.subplot(nrows, ncols, row*ncols + col + 1)
        img = plt.imread(img_paths[col])
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.title(class_name, fontsize=8)
plt.tight_layout()
plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_11156\875195845.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
     'kernel': ['rbf', 'poly'],
     
}

# Create a GridSearchCV object
model = GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=2,scoring='accuracy' )

# Choose a machine learning model
# model = RandomForestClassifier(random_state=0)

model.fit(data['training_data'], data['training_labels'])

# test performance (if using RandomForestClassifier)
# y_pred = model.predict(data['validation_data'])
# score = accuracy_score(y_pred, data['validation_labels'])
# print(score)

print(model.best_params_)
print(model.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.1min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.1min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.0min
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   9.2s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   9.4s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   8.5s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   8.4s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   9.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 1.1min
[CV] END .......................C=0.1, gamma=0.

In [12]:
with open('./flower.p', 'wb') as f:
    pickle.dump(model, f)
    f.close()

In [13]:
import pickle

from img2vec_pytorch import Img2Vec
from PIL import Image



with open('flower.p', 'rb') as f:
    model = pickle.load(f)

img2vec = Img2Vec()
class_labels = ['Daisy', 'Lavender', 'Lily', 'Rose','Sunflower']
image_path = './rose.jpg'

img = Image.open(image_path)

features = img2vec.get_vec(img)
features_2d = features.reshape(1, -1)

# Get prediction probabilities
prediction_probabilities = model.predict_proba(features_2d)[0]
for ind, prob in enumerate(prediction_probabilities):
    print(f'Class {class_labels[ind]}: {prob*100:.2f}%')
    
pred = model.predict([features])

print(pred) 

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: './rose.jpg'